# API Mapping version 1

In [1]:
from langchain_core.documents import Document
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_groq import ChatGroq
import requests, json, re
import os
import pandas as pd

In [ ]:
def create_api_mapping(cpp_code, java_methods_info):
     
     # Prepare the prompt
        prompt = f"""
You are an expert software engineer with good knowledge in C++ and Java .
You are given the following C++ method:
{cpp_code}


And the following top {len(java_methods_info)} candidate Java methods:
{java_methods_info}


Task:
1. Determine if these Java methods can together implement the exact behavior of the C++ method.
2. If yes, specify only those APIs are used, their relationship (AND/OR) between the java methods, and the flow or the order which methods follows which methods
3. If not, say "No exact match".
4. The mapping of java method and cpp code whether its one to one or one cpp to many java
5. And give the retrieved intent of the java methods that either standalone or together implement the exact behavior of the C++ method
Respond in structured JSON:
{{
"match": true/false,
"java_methods_used": [list of method names],
"api_flow": [list of APIs and relationships]
"Mapping" : one to one cpp java mapping or one cpp to many java method mapping
"Intent": intent of all selected java methods that implement exact behaviour of C++ methods
}}
"""

        api_key = os.environ["API_KEY"]
        response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        },
        data=json.dumps({
            "model": "qwen/qwen-2.5-coder-32b-instruct",
            "messages": [
                {"role": "system", "content": "You are a helpful expert software analyst."},
                {"role": "user", "content": prompt}
            ],
            "max_tokens":1024,
            "temperature": 0.2,
            
        })
        )
        result = response.json()
        print(result)
        return result["choices"][0]["message"]["content"]

In [ ]:
def intent_generator(code):

        # Prepare the prompt
        prompt = f"""
        Analyze the following C++ or java source code and write a very very short intent and how it performs that intent of the method.
        just two sentences. it should also include how it performs as operation
        Excepted Output:
        "Intent: What is does, Operation: How it does"

        C++ or java Source Code:
        {code}
        """

        # Build the request payload for a chat-based model (like Mistral or Falcon via OpenRouter)
        data = {
            "model": "mistralai/mixtral-8x7b-instruct",  # or another model available on OpenRouter
            "messages": [
                {"role": "system", "content": "You are a helpful expert software analyst."},
                {"role": "user", "content": prompt}
            ],
            "max_tokens": 1024,
            "temperature": 0.2
        }

        headers = {
            "Authorization": f"Bearer {os.environ['API_KEY']}",
            "Content-Type": "application/json"
        }

        response = requests.post('https://openrouter.ai/api/v1/chat/completions', json=data, headers=headers)

        if response.status_code == 200:
            result = response.json()
            # Extract generated text from the response structure
            description = result["choices"][0]["message"]["content"]
            return(description)
        else:
            return(f"Error: {response.status_code} - {response.text}")

In [ ]:
code = """public <T extends Number> double add(T a, T b) {
return a.doubleValue() + b.doubleValue();
}"""
result = intent_generator(code)
result

In [ ]:
code_list = [{'code': 'public void printMessage() {\nSystem.out.println("Hello, World!");\n}',
  'methodname': 'printMessage','classname':'',
  'intent': ' This is a Java method named "printMessage" which prints the string "Hello, World!" to the console when called. Its purpose is to display a simple greeting message.'},
 {'code': 'public int maxVal(int a, int b) {\nreturn (a > b) ? a : b;\n}',
  'methodname': 'maxVal','classname':'',
  'intent': ' This is a simple method in Java that takes two integer parameters, `a` and `b`. The method returns the maximum of the two input values by using a conditional operator. If `a` is greater than `b`, the method returns `a`, otherwise it returns `b`.'},
 {'code': 'public int add(int a) {\nreturn add(a, 10);\n}',
  'methodname': 'add','classname':'',
  'intent': ' This Java method, named "add", takes an integer "a" as an argument and calls the "add" method with arguments "a" and 10. The return type is an integer. The specific implementation of the add method is not provided, but it appears to perform some form of addition.'},
 {'code': 'public int add(int a, int b) {\nreturn a + b;\n}',
  'methodname': 'add','classname':'',
  'intent': ' This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the \'+\' operator.'},
 {'code': 'public double multiply(double a, double b) {\nreturn a * b;\n}',
  'methodname': 'multiply','classname':'',
  'intent': " This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase."},
 {'code': 'public int multiply(int a, int b) {\nreturn a * b;\n}',
  'methodname': 'multiply','classname':'',
  'intent': ' This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.'},
 {'code': 'public void increment(MutableInt x) {\nx.value++;\n}',
  'methodname': 'increment','classname':'MutableInt',
  'intent': ' This method, `increment`, takes a single parameter `x` of type `MutableInt`, and increments the value of `x` by 1. This method is likely to be used in situations where the state of a mutable integer object needs to be updated.'},
 {'code': 'class MutableInt {\npublic int value;\npublic MutableInt(int v) { value = v; }\n}',
  'methodname': '','classname':'MutableInt',
  'intent': ' The `MutableInt` class in C++ or Java is a simple container for an integer value. It provides a constructor to initialize the value and allows direct access to the value, making it mutable.'},
 {'code': 'class MathUtils {\npublic static int square(int x) {\nreturn x * x;\n}\n}',
  'methodname': 'square','classname':'MathUtils',
  'intent': ' The `square` method in the `MathUtils` class takes an integer `x` as an argument and returns the result of multiplying `x` by itself, which is the square of the input. This method is a simple utility for performing a square operation in a mathematical context.'},
 {'code': '\nclass Animal {\npublic void speak() {\nSystem.out.println("Animal sound");\n}\n}\n ',
  'methodname': 'speak','classname':'Animal',
  'intent': ' The `speak` method in the `Animal` class in the provided C++ or Java source code is a public method that prints the message "Animal sound" to the console. This method provides a simple way to make an abstract representation of an animal produce a sound, although it does not provide any specific sound for any particular animal species.'},
 {'code': ' \nclass Dog extends Animal {\n@Override\npublic void speak() {\nSystem.out.println("Woof!");\n}\n}\n',
  'methodname': 'speak','classname':'Animal, Dog',
  'intent': ' This is a Java method that overrides the speak() method in the Dog class, which is a subclass of the Animal class. When this method is called, it prints the string "Woof!" to the console, representing the sound that a dog makes.'},
 {'code': '\n public <T extends Number> double add(T a, T b) {\nreturn a.doubleValue() + b.doubleValue();\n}\n\n ',
  'methodname': 'add','classname':'Number',
  'intent': ' This method, with a generic type T that extends the Number class, is implemented in either Java or a compatible language. Its purpose is to add together two numeric values of the same type, returning the result as a double precision floating-point number.'},
 {'code': '\nclass FileHandler {\n// No automatic destructor; must use explicit method\npublic void close() {\nSystem.out.println("Cleaning up resources.");\n}\n@Override\nprotected void finalize() throws Throwable {\n// Called by GC, but not reliable\nclose();\n}\n}\n  ',
  'methodname': 'close','classname':'FileHandler',
  'intent': ' The `close()` method in the `FileHandler` class is used to clean up resources. There is no automatic destructor in this class, so developers must explicitly call `close()` to ensure proper resource cleanup. The `finalize()` method is overridden to call `close()`, but it is not reliable as it is dependent on the garbage collector.'},
 {'code': '\nclass Complex {\ndouble real, imag;\nComplex(double r, double i) { real = r; imag = i; }\npublic Complex add(Complex other) {\nreturn new Complex(real + other.real, imag + other.imag);\n}\n}\n ',
  'methodname': '','classname':'Complex',
  'intent': ' This code defines a simple `Complex` class in either C++ or Java (with minor modifications) to represent complex numbers. The `add` method takes a `Complex` number as an argument and returns a new `Complex` number representing the sum of the current and argument complex numbers.'},
  {'code': """public static double divide(double numerator, double denominator) {
        if (denominator == 0) {
            throw new ArithmeticException("Cannot divide by zero");
        }
        return numerator / denominator;
    }
""",
  'methodname': 'divide','classname':'','intent':'The function takes tow numbers and divides it'},
  {'code': """public class MathUtils {

    // Method to return the square root of a number
    public static double squareRoot(double number) {
        if (number < 0) {
            throw new IllegalArgumentException("Cannot compute square root of a negative number.");
        }
        return Math.sqrt(number);
    }
""",
  'methodname': 'squareRoot','classname':'MathUtils','intent':'The function takes a number and gives the square root of it'}]

In [ ]:
for code in code_list:
    code['intent'] = intent_generator(code['code'])

In [ ]:
def create_documents_from_chunks(chunk_list):
    list_of_documents = []
    for chunk in chunk_list:
        document = Document(
            page_content=f"code: {chunk['code']},'intent': {chunk['intent']},methods: {chunk['methodname']},classname: {chunk['classname']}",
        )
        list_of_documents.append(document)
    return list_of_documents
java_method_documents = create_documents_from_chunks(code_list)

In [ ]:
for i in java_method_documents:
    print(i.page_content)

In [ ]:
embedding = HuggingFaceEmbeddings(
            model_name="intfloat/multilingual-e5-base",
            model_kwargs={"device": "cpu"})
vector_store = FAISS(
            embedding_function=embedding,
            index=faiss.IndexFlatL2(len(embedding.embed_query("hello world"))),
            docstore=InMemoryDocstore(),
            index_to_docstore_id={},
        )

In [ ]:
vector_store.add_documents(documents=java_method_documents)

In [ ]:
list_of_dict_of_mapping = []

In [ ]:
# Example 10
code = """
 class Complex {
public:
double real, imag;
Complex(double r, double i): real(r), imag(i) {}
Complex operator+(const Complex& other) {
return Complex(real + other.real, imag + other.imag);
}
};
"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])
list_of_dict_of_mapping.append(my_dict)

In [ ]:
# example 3
code = """
 C++
int add(int a, int b = 10) {
return a + b;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])
list_of_dict_of_mapping.append(my_dict)

In [ ]:
# Example 5
code = """
void increment(int &x) {
x++;
}

"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])
list_of_dict_of_mapping.append(my_dict)

In [ ]:
# example 7
code = """
class FileHandler {
public:
~FileHandler() {
// Automatically called when object goes out of scope
std::cout << "Cleaning up resources." << std::endl;
}
};

"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])
list_of_dict_of_mapping.append(my_dict)


In [ ]:
# example 7
code = """
class Animal {
public:
virtual void speak() {
std::cout << "Animal sound" << std::endl;
}
};
class Dog : public Animal {
public:
void speak() override {
std::cout << "Woof!" << std::endl;
}
};

"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])
list_of_dict_of_mapping.append(my_dict)

In [ ]:
df = pd.DataFrame(list_of_dict_of_mapping)
print(df)

# API MAPPING VERSION 2

In [2]:
def create_api_mapping(cpp_code, java_methods_info):
     
     # Prepare the prompt
        prompt = f"""
You are an expert software engineer with deep knowledge of C++ and Java.

C++ method:
{cpp_code}

Candidate Java methods (with intents):
{java_methods_info}

Task:
1. Analyze the C++ method's behavior (not just the code, but its intent and required operations).
2. Identify which Java methods individually OR in combination can reproduce this behavior.
   - If no single method exists, check if multiple retrieved methods can be composed (e.g., method1() + method2()).
3. For each potential match:
   - List the methods used.
   - Describe the relationship: AND (must be used together) or OR (can be used interchangeably).
   - Provide the execution flow/sequence.
4. If no methods (individually or combined) can implement the behavior, say "No exact match".
5. Output the result as structured JSON:
{{
  "match": true/false,
  "java_methods_used": [list of method names],
  "api_flow": ["sequence of methods with relationships"],
  "Mapping": "one-to-one" or "one-to-many",
  "Intent": "Summarized intent of the selected methods that together replicate the C++ behavior"
}}
"""

        api_key = os.environ["API_KEY"]
        response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        },
        data=json.dumps({
            "model": "qwen/qwen-2.5-coder-32b-instruct",
            "messages": [
                {"role": "system", "content": "You are a helpful expert software analyst."},
                {"role": "user", "content": prompt}
            ],
            "max_tokens":1024,
            "temperature": 0.2,
            
        })
        )
        result = response.json()
        print(result)
        return result["choices"][0]["message"]["content"]

In [3]:
def intent_generator(code):

        # Prepare the prompt
        prompt = f"""
        Analyze the following C++ or java source code and write a very very short intent and how it performs that intent of the method.
        just two sentences. it should also include how it performs as operation
        Excepted Output:
        "Intent: What is does, Operation: How it does"

        C++ or java Source Code:
        {code}
        """

        # Build the request payload for a chat-based model (like Mistral or Falcon via OpenRouter)
        data = {
            "model": "mistralai/mixtral-8x7b-instruct",  # or another model available on OpenRouter
            "messages": [
                {"role": "system", "content": "You are a helpful expert software analyst."},
                {"role": "user", "content": prompt}
            ],
            "max_tokens": 1024,
            "temperature": 0.2
        }

        headers = {
            "Authorization": f"Bearer {os.environ['API_KEY']}",
            "Content-Type": "application/json"
        }

        response = requests.post('https://openrouter.ai/api/v1/chat/completions', json=data, headers=headers)

        if response.status_code == 200:
            result = response.json()
            # Extract generated text from the response structure
            description = result["choices"][0]["message"]["content"]
            return(description)
        else:
            return(f"Error: {response.status_code} - {response.text}")

In [4]:
code_list = [{'methodname': 'printMessage','classname':'',
  'intent': ' This is a Java method named "printMessage" which prints the string "Hello, World!" to the console when called. Its purpose is to display a simple greeting message.'},
 {'methodname': 'maxVal','classname':'',
  'intent': ' This is a simple method in Java that takes two integer parameters, `a` and `b`. The method returns the maximum of the two input values by using a conditional operator. If `a` is greater than `b`, the method returns `a`, otherwise it returns `b`.'},
 {'methodname': 'add','classname':'',
  'intent': ' This Java method, named "add", takes an integer "a" as an argument and calls the "add" method with arguments "a" and 10. The return type is an integer. The specific implementation of the add method is not provided, but it appears to perform some form of addition.'},
 {'methodname': 'add','classname':'',
  'intent': ' This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the \'+\' operator.'},
 {'methodname': 'multiply','classname':'',
  'intent': " This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase."},
 {'methodname': 'multiply','classname':'',
  'intent': ' This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.'},
 {'methodname': 'increment','classname':'MutableInt',
  'intent': ' This method, `increment`, takes a single parameter `x` of type `MutableInt`, and increments the value of `x` by 1. This method is likely to be used in situations where the state of a mutable integer object needs to be updated.'},
 {'methodname': '','classname':'MutableInt',
  'intent': ' The `MutableInt` class in C++ or Java is a simple container for an integer value. It provides a constructor to initialize the value and allows direct access to the value, making it mutable.'},
 {'methodname': 'square','classname':'MathUtils',
  'intent': ' The `square` method in the `MathUtils` class takes an integer `x` as an argument and returns the result of multiplying `x` by itself, which is the square of the input. This method is a simple utility for performing a square operation in a mathematical context.'},
 {'methodname': 'speak','classname':'Animal',
  'intent': ' The `speak` method in the `Animal` class in the provided C++ or Java source code is a public method that prints the message "Animal sound" to the console. This method provides a simple way to make an abstract representation of an animal produce a sound, although it does not provide any specific sound for any particular animal species.'},
 {'methodname': 'speak','classname':'Animal, Dog',
  'intent': ' This is a Java method that overrides the speak() method in the Dog class, which is a subclass of the Animal class. When this method is called, it prints the string "Woof!" to the console, representing the sound that a dog makes.'},
 {'methodname': 'add','classname':'Number',
  'intent': ' This method, with a generic type T that extends the Number class, is implemented in either Java or a compatible language. Its purpose is to add together two numeric values of the same type, returning the result as a double precision floating-point number.'},
 {'methodname': 'close','classname':'FileHandler',
  'intent': ' The `close()` method in the `FileHandler` class is used to clean up resources. There is no automatic destructor in this class, so developers must explicitly call `close()` to ensure proper resource cleanup. The `finalize()` method is overridden to call `close()`, but it is not reliable as it is dependent on the garbage collector.'},
 {'methodname': '','classname':'Complex',
  'intent': ' This code defines a simple `Complex` class in either C++ or Java (with minor modifications) to represent complex numbers. The `add` method takes a `Complex` number as an argument and returns a new `Complex` number representing the sum of the current and argument complex numbers.'},
  {'methodname': 'divide','classname':'','intent':'The function takes tow numbers and divides it'},
  {'methodname': 'squareRoot','classname':'MathUtils','intent':'The function takes a number and gives the square root of it'}]

In [5]:
def create_documents_from_intents(chunk_list):
    list_of_documents = []
    for chunk in chunk_list:
        document = Document(
            page_content=f"'intent': {chunk['intent']},methods: {chunk['methodname']},classname: {chunk['classname']}",
            metadata = {'methods': chunk['methodname'],'classname': chunk['classname']}
        )
        list_of_documents.append(document)
    return list_of_documents
only_intent_document = create_documents_from_intents(code_list)

In [6]:
embedding = HuggingFaceEmbeddings(
            model_name="intfloat/multilingual-e5-base",
            model_kwargs={"device": "cpu"})
vector_store1 = FAISS(
            embedding_function=embedding,
            index=faiss.IndexFlatL2(len(embedding.embed_query("hello world"))),
            docstore=InMemoryDocstore(),
            index_to_docstore_id={},
        )
vector_store1.add_documents(only_intent_document)

C:\Users\soham\AppData\Local\Temp\ipykernel_15284\4282650126.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding = HuggingFaceEmbeddings(
c:\c\Users\Hari\VISTAR_PROJECT\SM7to8_api_knowledge_mapping\myenv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


['ffc73d95-46d2-4ecc-9471-94bb55d0c543',
 '3bd7630a-3623-4e82-a9e6-ae9609d5b41b',
 '095523cc-bf4e-4587-82bb-133ff19d9c22',
 'd41011b7-27be-4a7d-84e8-c37459967e8d',
 'f3f5bf41-8339-437e-bfc6-dda5d2be7c5b',
 '26a691de-c725-4512-a9fe-9eb83b5e60b4',
 'f4bc2202-e5b6-42c3-9e90-99e9d1d397e3',
 '83a389d3-4f62-4ffe-89df-452aa1a46996',
 'f12513ac-0d93-4566-beeb-bba5c8491af7',
 '107c3d8b-6a72-46df-92e9-50c3fab89eef',
 '44b37920-1fd9-4616-86b1-b67fbae7e8c5',
 'cdec4eb7-e37c-4427-bdf6-3514b5753954',
 '4b68c30b-8dbc-4593-a260-f0dc2ff37173',
 'fa4cf492-2985-40ef-85d8-78aa372bd8c0',
 '08938ece-c577-46d1-93d9-79dda9598dbd',
 '9abcd6da-8e01-4ed0-be87-42f8f45ab808']

In [17]:
# Example Mean:
code = """
 int mean(int a, int b) {
    return (a + b) / 2.0;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: Calculates the mean (average) of two integers, Operation: By adding the two integers, dividing the sum by 2.0, and returning the result as a floating point number.
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: 
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: 
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: 
* 'intent': The function takes a number and gives the

In [14]:
# Example Root Mean:
code = """
 int rootMeanSquare(int a, int b) {
    return std::sqrt((a * a + b * b) / 2.0);
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: Calculates the root mean square of two integers, providing a measure of their average magnitude.
Operation: Performs integer multiplication, addition, and division, then obtains the square root of the result using a floating-point number.
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: 
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: 
* 'intent':  This is a Java method named "add" that takes two integer par

In [10]:
# Example geomentric Mean:
code = """
 double geometricMean(double a, double b) {
    return std::sqrt(a * b);
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: Calculate the geometric mean of two numbers, Operation: It multiplies the numbers together, then takes the square root of the result.
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: 
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: 
* 'intent': The function takes tow numbers and divides it,methods: divide,classname: 
{'id': 'gen-1756457037-KX3AjObKIl2jCw0VfH2z', 'provider': 'Nebius', 'model': 'qwen/qwen-2.5-

In [11]:
# Example harmonic mean
code = """
 double harmonicMean(double a, double b) {
    return (2.0 * a * b) / (a + b);
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: Calculates the harmonic mean of two numbers, Operation: Performs this calculation by invoking the formula (2 * a * b) / (a + b) using given numbers a and b.
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: 
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: 
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: 
* 'intent': The function takes a number and gives the square

In [15]:
# Example variance
code = """
double variance(double a, double b) {
    double mean = (a + b) / 2.0;
    return ((a - mean) * (a - mean) + (b - mean) * (b - mean)) / 2.0;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: Calculate the variance of two numbers, Operation: By finding the mean of the two numbers, then calculating the difference of each number from the mean, squaring those differences, averaging the squared differences, and returning the result.
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils
* 'intent': The function takes tow numbers and divides it,methods: divide,classname: 
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: 
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multip

In [13]:
# Example Standard Deviation
code = """
 double standardDeviation(double a, double b) {
    double mean = (a + b) / 2.0;
    double variance = ((a - mean) * (a - mean) + (b - mean) * (b - mean)) / 2.0;
    return std::sqrt(variance);
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: Calculates the standard deviation of two numbers, operation: By first finding the mean, then calculating the variance, and finally returning the square root of the variance.
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: 
* 'intent': The function takes tow numbers and divides it,methods: divide,classname: 
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: 
{'id': 'gen-1756457423-PSkjvEBrVOxtMLDfwyvS', 'provi

In [ ]:
# Example Default argument
code = """
 int add(int a, int b = 10) {
return a + b;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: The `add` method returns the sum of two integers.
Operation: It performs addition of the first argument 'a' with the second argument 'b' having a default value of 10.
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: 
* 'intent':  This Java method, named "add", takes an integer "a" as an argument and calls the "add" method with arguments "a" and 10. The return type is an integer. The specific implementation of the add method is not provided, but it appears to perform some form of addition.,methods: add,classname: 
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: 
* 'intent':  This code defines a simple `Complex` class in either C++ or Java (with minor modifications) to repr

In [19]:
# Example Pass by reference

code = """
 void increment(int &x) {
x++;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: Increments the value of a passed integer variable, Operation: Performs this operation in-place by directly accessing and modifying the memory location of the variable.
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: 
* 'intent':  This method, `increment`, takes a single parameter `x` of type `MutableInt`, and increments the value of `x` by 1. This method is likely to be used in situations where the state of a mutable integer object needs to be updated.,methods: increment,classname: MutableInt
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: 
{'id': 'gen-1756459

In [22]:
# Example Reference

code = """
 void increment(int &x) {
x++;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: Increments the value of a given integer, Operation: Performs this increment directly on the integer reference provided as an argument, using the postfix increment operator '++'.
* 'intent':  This method, `increment`, takes a single parameter `x` of type `MutableInt`, and increments the value of `x` by 1. This method is likely to be used in situations where the state of a mutable integer object needs to be updated.,methods: increment,classname: MutableInt
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: 
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: 
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils
{'id': 'g

In [ ]:
# example File Handler example
code = """
class FileHandler {
public:
~FileHandler() {
// Automatically called when object goes out of scope
std::cout << "Cleaning up resources." << std::endl;
}
};

"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])


* 'intent':  The `close()` method in the `FileHandler` class is used to clean up resources. There is no automatic destructor in this class, so developers must explicitly call `close()` to ensure proper resource cleanup. The `finalize()` method is overridden to call `close()`, but it is not reliable as it is dependent on the garbage collector.,methods: close,classname: FileHandler
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils
* 'intent':  This is a Java method named "printMessage" which prints the string "Hello, World!" to the console when called. Its purpose is to display a simple greeting message.,methods: printMessage,classname: 
* 'intent': The function takes tow numbers and divides it,methods: divide,classname: 
{'id': 'gen-1756460411-5piIzBOICOU2i658bngH', 'provider': 'DeepInfra', 'model': 'qwen/qwen-2.5-coder-32b-instruct', 'object': 'chat.completion', 'created': 1756460411, 'choices': [{'logprobs': None, 'finish_

In [24]:
# example 7
code = """
class Complex {
public:
double real, imag;
Complex(double r, double i): real(r), imag(i) {}
Complex operator+(const Complex& other) {
return Complex(real + other.real, imag + other.imag);
}
};

"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])


* 'intent':  This code defines a simple `Complex` class in either C++ or Java (with minor modifications) to represent complex numbers. The `add` method takes a `Complex` number as an argument and returns a new `Complex` number representing the sum of the current and argument complex numbers.,methods: ,classname: Complex
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: 
* 'intent':  This method, with a generic type T that extends the Number class, is implemented in either Java or a compatible language. Its purpose is to add together two numeric values of the same type, returning the result as a double precision floating-point number.,methods: add,classname: Number
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result